In [1]:
import pandas as pd
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.corpus import CorpusReader
import re
import numpy as np
import zipfile
import io

# Loading data

In [2]:
reader = CorpusReader(Loc.corpus_path/'lenta.base.zip')

In [3]:
frames = reader.get_frames()

In [5]:
df = frames.first()

In [6]:
df.head(5)

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
195000,195000,195000,0,195000,1,Организатор,ru,11,0,0,0,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip
195001,195001,195000,1,195000,1,экстремального,ru,14,1,0,0,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip
195002,195002,195000,2,195000,1,аттракциона,ru,11,2,0,0,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip
195003,195003,195000,3,195000,1,в,ru,1,3,0,0,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip
195004,195004,195000,4,195000,0,Карачаево-Черкесии,ru,18,4,0,0,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip


# Чтобы что бы

In [21]:
def get_chto_by(df):
    what = df[df['word'] == 'что']
    what_next = what[['sentence_id', 'word_index']].copy()
    what_next['word_index'] += 1
    what_neighbour = df.merge(what_next, on=['sentence_id', 'word_index'], how='inner')
    
    by = what_neighbour[what_neighbour['word'] == 'бы']
    
    return pd.concat([what.merge(by['word_id'] - 1, on=['word_id'], how='inner'), by]).sort_values(['word_id'])

In [22]:
chto_by = get_chto_by(df)
chto_by

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id,original_corpus_id,is_target
0,241295,197321,22,195180,1,что,ru,3,46295,2321,180,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,True
438,241296,197321,23,195180,1,бы,ru,2,46296,2321,180,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False


In [67]:
def get_targets(df):
    targets = df.set_index('word_id').word.str.lower() == 'чтобы'

    chto = df[df['word'] == 'что']
    chto_next = chto['word_id'] + 1
    chto_neighbour = df.merge(chto_next, how='inner')
    by = chto_neighbour[chto_neighbour['word'] == 'бы']

    targets[by['word_id'] - 1] = True

    return targets.values

Negative sample building

In [39]:
result = df.copy()
chtoby = result['word'] == 'чтобы'

In [40]:
# transforming 'что' + 'бы' to 'чтобы'
chto = result[result['word'] == 'что']
chto_next = chto[['sentence_id', 'word_index']].copy()
chto_next['word_index'] += 1
chto_neighbor = result.merge(chto_next, on=['sentence_id', 'word_index'], how='inner')
by = chto_neighbor[chto_neighbor['word'] == 'бы']

print(by.shape)

if by.shape[0] != 0:
    chto_with_pair_loc = by['word_id']
    result.loc[chto_with_pair_loc - 1, 'word'] = 'чтобы'
    result.loc[chto_with_pair_loc - 1, 'word_length'] += 3

    result = result.drop(chto_with_pair_loc)

(1, 14)


In [41]:
from tg.grammar_ru.common.separator import Separator
# transforming 'чтобы' to 'что' + 'бы'
result.loc[chtoby, 'word'] = 'что бы'

In [42]:
result = Separator.separate_string(Separator.Viewer().to_text(result))

Building index

In [14]:
import numpy as np
from tg.grammar_ru.ml.tasks.train_index_builder.index_builders import ChtobyIndexBuilder

In [15]:
import importlib
from tg.grammar_ru.ml.tasks.train_index_builder import index_builders

ChtobyIndexBuilder = importlib.reload(index_builders).ChtobyIndexBuilder

In [16]:
builder = ChtobyIndexBuilder()

In [17]:
index = builder.build_train_index(df)

In [18]:
positive, negative = index

In [23]:
positive

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id
197286,197286,195112,0,195007,1,Я,ru,1,2286,112,7,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,0
197287,197287,195112,1,195007,1,чувствую,ru,8,2287,112,7,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,0
197288,197288,195112,2,195007,1,себя,ru,4,2288,112,7,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,0
197289,197289,195112,3,195007,0,так,ru,3,2289,112,7,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,0
197290,197290,195112,4,195007,1,",",punct,1,2290,112,7,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428867,428867,206927,22,195926,1,и,ru,1,233867,11927,926,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,158
428868,428868,206927,23,195926,1,исключить,ru,9,233868,11927,926,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,158
428869,428869,206927,24,195926,1,возможность,ru,11,233869,11927,926,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,158
428870,428870,206927,25,195926,0,провокации,ru,10,233870,11927,926,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,False,0,158


In [57]:
negative = builder._build_negative_from_positive(positive)

In [59]:
negative[negative['word'] == 'чтобы']

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id
241295,241295,197321,22,195180,1,чтобы,ru,6,46295,2321,180,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,True,1,27


In [45]:
neg[neg['word'] == 'чтобы']

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id


In [16]:
pos, neg = ChtobyIndexBuilder().build_train_index(df)

In [17]:
neg[neg['word'] == 'чтобы']

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id
241295,241295,197321,22,195180,1,чтобы,ru,6,46295,2321,180,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip,lenta.base.zip,True,1,27


# Training

In [1]:
from pathlib import Path

from tg.grammar_ru.common import Loc
from research.common import run_training


bundle_name = 'big'
bundle_path = Loc.bundles_path/f'chtoby/{bundle_name}'

In [2]:
result = run_training.run_local(bundle_path)

2022-11-23 09:55:20.780737+00:00 INFO: Training starts. Info: {'name': 'TSAG-'}
2022-11-23 09:55:20.871025+00:00 INFO: Ensuring/loading bundle. Bundle before:
{'slovnet': {'shape': (6747861, 18), 'index_name': 'word_id'}, 'syntax_fixes': {'shape': (6747861, 4), 'index_name': 'word_id'}, 'syntax_stats': {'shape': (6747861, 6), 'index_name': 'word_id'}, 'index': {'shape': (266027, 5), 'index_name': 'sample_id'}, 'syntax_closure': {'shape': (18338761, 4), 'index_name': 'entry_id'}, 'src': {'shape': (6747861, 18), 'index_name': None}, 'pymorphy': {'shape': (6747861, 16), 'index_name': 'word_id'}}
2022-11-23 09:55:20.899514+00:00 INFO: Bundle loaded
{'slovnet': {'shape': (6747861, 18), 'index_name': 'word_id', 'columns': ['POS', 'Case', 'Degree', 'Gender', 'Number', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_fixes': {'shape': (6747861, 4), 'index_name': 'word_id', 'columns': ['syntax_parent_id', 'root', 'cycle_status', 'correct_root'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_stats'

KeyboardInterrupt: 

# Debug

In [4]:
reader = CorpusReader(Loc.bundles_path/'chtoby/prepare/raw/raw.zip')

In [5]:
frames = reader.get_frames()

In [6]:
frame = frames.first()

In [7]:
frame

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id,original_word_id,original_sentence_id,original_paragraph_id,updated
0,0,0,0,0,1,Товарищеская,ru,12,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,0.0,1242,65,9,False
1,1,0,1,0,1,встреча,ru,7,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,0.0,1243,65,9,False
2,2,0,2,0,1,прошла,ru,6,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,0.0,1244,65,9,False
3,3,0,3,0,1,в,ru,1,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,0.0,1245,65,9,False
4,4,0,4,0,1,присутствии,ru,11,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,0.0,1246,65,9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50100,50100,47736,21,47728,1,",",punct,1,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,997.0,1439196,1317013,1311705,False
50101,50101,47736,22,47728,1,чтобы,ru,5,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,True,0,997.0,1439197,1317013,1311705,False
50102,50102,47736,23,47728,1,потрясти,ru,8,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,997.0,1439198,1317013,1311705,False
50103,50103,47736,24,47728,0,нигерийца,ru,9,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,lenta.base.zip,False,0,997.0,1439199,1317013,1311705,False


In [8]:
frame[frame['label'] == 1].head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id,original_corpus_id,is_target,label,reference_sentence_id,original_word_id,original_sentence_id,original_paragraph_id,updated
4993,4993,4993,0,4993,1,Товарищеская,ru,12,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,None,False,1,NaN,0,0,0,False
4994,4994,4993,1,4993,1,встреча,ru,7,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,None,False,1,NaN,1,0,0,False
4995,4995,4993,2,4993,1,прошла,ru,6,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,None,False,1,NaN,2,0,0,False
4996,4996,4993,3,4993,1,в,ru,1,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,None,False,1,NaN,3,0,0,False
4997,4997,4993,4,4993,1,присутствии,ru,11,2917e260-0518-49a5-8b7b-75b7af3f7891,raw.zip,None,False,1,NaN,4,0,0,False
